In [1]:
!pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers
!pip install -q -U git+https://github.com/huggingface/peft.git

import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
model_id = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [3]:
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [5]:
def generate_prompt(data_point):
  return f"""
<Human>: {data_point["question"]}
<AI>: {data_point["answer"]}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt


In [15]:
from datasets import load_dataset
dataset_name = 'microsoft/orca-math-word-problems-200k'
dataset = load_dataset(dataset_name, split="train[:7000]")

dataset = dataset.shuffle().map(generate_and_tokenize_prompt)


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y'
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to t

In [17]:
training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir="./falcon-qa-finetuning",
    save_strategy='epoch',
    push_to_hub=True
)
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
10,0.701700
20,0.764700
30,0.745500


Step,Training Loss
10,0.575100
20,0.648100
30,0.668800
40,0.704600
50,0.731500
60,0.630200
70,0.770800
80,0.692400
90,0.775800
100,0.684600


Step,Training Loss
10,0.535800
20,0.523600
30,0.587500
40,0.573300
50,0.592200
60,0.599800
70,0.578500
80,0.638000
90,0.633800
100,0.654000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=7000, training_loss=0.6163497987474714, metrics={'train_runtime': 6329.8205, 'train_samples_per_second': 2.212, 'train_steps_per_second': 1.106, 'total_flos': 1.8878674645522944e+17, 'train_loss': 0.6163497987474714, 'epoch': 2.0})

In [ ]:
from transformers import AutoModelForCausalLM

# Model identifier for the fine-tuned model
fine_tuned_model_id = "./falcon-qa-finetuning"

# Load the fine-tuned model
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_id)

# Ensure the model is in evaluation mode
fine_tuned_model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
!zip -r falcon_qa_finetuning.zip falcon-qa-finetuning


  adding: falcon-qa-finetuning/ (stored 0%)
  adding: falcon-qa-finetuning/runs/ (stored 0%)
  adding: falcon-qa-finetuning/runs/Mar14_12-51-45_672eac663d46/ (stored 0%)
  adding: falcon-qa-finetuning/runs/Mar14_12-51-45_672eac663d46/events.out.tfevents.1710420707.672eac663d46.480.1 (deflated 63%)
  adding: falcon-qa-finetuning/runs/Mar14_12-52-23_672eac663d46/ (stored 0%)
  adding: falcon-qa-finetuning/runs/Mar14_12-52-23_672eac663d46/events.out.tfevents.1710420744.672eac663d46.480.2 (deflated 75%)
  adding: falcon-qa-finetuning/runs/Mar14_12-26-30_672eac663d46/ (stored 0%)
  adding: falcon-qa-finetuning/runs/Mar14_12-26-30_672eac663d46/events.out.tfevents.1710419191.672eac663d46.480.0 (deflated 74%)
  adding: falcon-qa-finetuning/runs/Mar14_13-05-35_672eac663d46/ (stored 0%)
  adding: falcon-qa-finetuning/runs/Mar14_13-05-35_672eac663d46/events.out.tfevents.1710421536.672eac663d46.480.4 (deflated 73%)
  adding: falcon-qa-finetuning/runs/Mar14_13-05-11_672eac663d46/ (stored 0%)
  addi

In [4]:
!huggingface-cli login
!huggingface-cli upload ./falcon-qa-finetuning.zip



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [7]:
!huggingface-cli upload falcon_qa_finetuning.zip


Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
falcon_qa_finetuning.zip: 100% 121M/121M [00:03<00:00, 37.0MB/s]
https://huggingface.co/devesh-2002/falcon_qa_finetuning.zip/blob/main/falcon_qa_finetuning.zip


In [9]:
!zip -r checkpoint-7000.zip /content/falcon-qa-finetuning/checkpoint-7000


  adding: content/falcon-qa-finetuning/checkpoint-7000/ (stored 0%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/rng_state.pth (deflated 25%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/scheduler.pt (deflated 55%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/adapter_config.json (deflated 51%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/adapter_model.safetensors (deflated 7%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/trainer_state.json (deflated 85%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/training_args.bin (deflated 51%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/README.md (deflated 66%)
  adding: content/falcon-qa-finetuning/checkpoint-7000/optimizer.pt (deflated 9%)


In [10]:
from google.colab import files
files.download('checkpoint-7000.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>